In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import os
import random
import json
import glob

In [ ]:
# Install nb_black for autoformatting
!pip install nb_black --quiet
%load_ext lab_black

In [ ]:
# Constante
ROWS_PER_FRAME = 543
data_dir = "/kaggle/input/full-train-set-algo/full_train_set_algo"
output_dir = "/kaggle/working/"
landmark_fimes_dir = (
    "/kaggle/input/full-train-set-algo/full_train_set_algo/processed_with_algo"
)

In [ ]:
def seed_it_all(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


seed_it_all()  # Reproducible

In [ ]:
# ROWS_PER_FRAME = 543  # number of landmarks per frame
def load_relevant_data_subset(pq_path):
    data_columns = ["x", "y", "z"]
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [ ]:
def read_json(path):
    with open(path, "r") as file:
        json_data = json.load(file)
    return json_data

In [ ]:
train_df = pd.read_csv(data_dir + "/train.csv")
train_df["path"] = data_dir + "/" + train_df["path"]
display(train_df.head(2)), len(train_df)

In [ ]:
train_df.describe()

In [ ]:
train_df.tail()

In [ ]:
train_df.hist

In [ ]:
train_df.info()

In [ ]:
csv_file_path = "/kaggle/input/full-train-set-algo/full_train_set_algo/train.csv"

# Read the 'sign' column from 'train.csv'
df_train = pd.read_csv(csv_file_path, usecols=["sign"])

# Get the unique signs and sort them alphabetically
unique_signs = sorted(df_train["sign"].unique())

# Create a dictionary mapping sign names to a unique index
sign_to_index_map = {sign: index for index, sign in enumerate(unique_signs)}

# Convert the dictionary to a JSON object with a root element
json_object = json.dumps({"root": sign_to_index_map}, indent=4)

# Write the JSON object to a file
json_file_path = "sign_to_prediction_index_map.json"
with open(json_file_path, "w") as json_file:
    json_file.write(json_object)

# Output the path to the JSON file (for information purposes)
print(f"JSON file created at: {json_file_path}")

In [ ]:
s2p_map = read_json(os.path.join(output_dir, "sign_to_prediction_index_map.json"))
s2p_map = s2p_map["root"]
p2s_map = {v: k for k, v in s2p_map.items()}

encoder = lambda x: s2p_map.get(x)
decoder = lambda x: p2s_map.get(x)

train_df["label"] = train_df["sign"].map(encoder)
print(f"shape = {train_df.shape}")

train_df.head(2)

In [ ]:
participants = os.listdir(landmark_fimes_dir)
print(f"Total number of participants = {len(participants)}")
print(
    f"Average number of sequences per participant = {len(glob.glob(landmark_fimes_dir + '/*/*.parquet'))/len(participants)}"
)

In [ ]:
int(21 * 4498.9), train_df.shape[0]  # ~ same

In [ ]:
sample_path = train_df.path[0]
sample = pd.read_parquet(sample_path)

print(f"Sample shape = {sample.shape}")
print(f"Number of Frames = {int(len(sample) / ROWS_PER_FRAME)}")
# ROWS_PER_FRAME = 543 i.e. one frame is represented by 543 row in our dataset, including the face, both hands and pose
# n_frame can also be found : 20->42=>23frames i.e. sample.frame.max() -> sample.frame.min() : sample.nunique()

display(sample), display(sample.iloc[541:544])

In [ ]:
sample.isna().sum()  # probably the hand as explain in the 'Dataset Description' section

In [ ]:
sample_data_np = load_relevant_data_subset(sample_path)
print(f"shape = {sample_data_np.shape} = (n_frames, row_per_frame, xyz) \n")
sample_data_np[:1, :4, :]

In [ ]:
sample_path_hand = train_df.path[5]  # 5 : empirical choice
sample_for_hand = pd.read_parquet(sample_path_hand)

print(f"Number of Frames = {int(len(sample_for_hand) / ROWS_PER_FRAME)}")
print(f"First frame indice is {sample_for_hand.frame.min()}")
print(f"Last frame indice is {sample_for_hand.frame.max()}")
print(f"Sample signe is : {train_df.sign[5]}")

right_hand_sample = sample_for_hand[sample_for_hand.type == "right_hand"]
left_hand_sample = sample_for_hand[sample_for_hand.type == "left_hand"]
right_hand_sample.head(2)

In [ ]:
print(
    f"Percentage of nulls in Right Hand data = {100*np.mean(right_hand_sample['x'].isnull()):.2f} %"
)
print(
    f"Percentage of nulls in Left Hand data = {100*np.mean(left_hand_sample['x'].isnull()):.02f} %"
)

In [ ]:
edges = [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (0, 5),
    (0, 17),
    (5, 6),
    (6, 7),
    (7, 8),
    (5, 9),
    (9, 10),
    (10, 11),
    (11, 12),
    (9, 13),
    (13, 14),
    (14, 15),
    (15, 16),
    (13, 17),
    (17, 18),
    (18, 19),
    (19, 20),
]  # see above


def plot_frame(df, frame_id, ax):
    df = df[df.frame == frame_id].sort_values(["landmark_index"])
    x = list(df.x)
    y = list(df.y)

    ax.scatter(df.x, df.y, color="dodgerblue")
    for i in range(len(x)):
        ax.text(x[i], y[i], str(i))

    for edge in edges:
        ax.plot([x[edge[0]], x[edge[1]]], [y[edge[0]], y[edge[1]]], color="salmon")
        ax.set_title(f"Frame no. {frame_id}")
        ax.axis(False)


def plot_frame_seq(df, frame_id_range, n_frames):
    frames = np.linspace(
        frame_id_range[0], frame_id_range[1], n_frames, dtype=int, endpoint=True
    )
    fig, ax = plt.subplots(n_frames, 1, figsize=(5, 25))
    for i in range(n_frames):
        plot_frame(df, frames[i], ax[i])

    plt.show()

In [ ]:
# plot_frame_seq(right_hand_sample, (20, 40), 5)  # take 1 frame out of 4

In [ ]:
class FeatureGen(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, x):
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x = np.mean(x, axis=0)
        return x


feature_converter = FeatureGen()

In [ ]:
data_lenght_experiment = len(train_df)
data_lenght_experiment

In [ ]:
def convert_row(row):
    x = load_relevant_data_subset(os.path.join("/kaggle/input/asl-signs", row.path))
    x = feature_converter(x)
    return x, row.label

In [ ]:
def convert_and_save_data():
    np_features = np.zeros((data_lenght_experiment, ROWS_PER_FRAME, 3))
    np_labels = np.zeros(data_lenght_experiment)

    print(f"Total data to processe : {data_lenght_experiment}")
    for index, row in tqdm(train_df.iterrows()):
        if index > data_lenght_experiment - 1:
            break

        data = load_relevant_data_subset(row.path)
        feature, label = convert_row(row)
        np_features[index, :, :] = feature
        np_labels[index] = label

    np.save("features.npy", np_features)
    np.save("labels.npy", np_labels)

In [ ]:
try:
    features = np.load("/kaggle/working/feature.npy")
    labels = np.load("/kaggle/working/label.npy")
except:
    convert_and_save_data()

In [ ]:
features = np.load("features.npy")
labels = np.load("labels.npy")

In [ ]:
def get_model(n_labels=39, learning_rate=0.001):
    inputs = layers.Input(shape=(ROWS_PER_FRAME, 3))
    x = layers.Dense(128, activation="relu")(inputs)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dense(8, activation="relu")(x)
    x = layers.Flatten()(x)
    output = layers.Dense(n_labels, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["accuracy"],
    )

    return model

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "./ASL_model",
    save_best_only=True,
    restore_best_weights=True,
    monitor="val_accuracy",
    mode="max",
    verbose=False,
)

cb_list = [checkpoint_callback]

X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.2, stratify=labels, random_state=42
)

model = get_model()
model.summary()

In [ ]:
try:
    model = tf.keras.models.load_model("./ASL_mode")
except:
    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=200,
        callbacks=cb_list,
        batch_size=12,
    )

In [ ]:
model = tf.keras.models.load_model("./ASL_model")
score = model.evaluate(X_val, y_val)

In [ ]:
def get_inference_model(model):
    inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
    x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
    x = tf.reduce_mean(x, axis=0, keepdims=True)
    x = model(x)
    output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output)
    inference_model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"]
    )
    return inference_model

In [ ]:
inference_model = get_inference_model(model)
inference_model.summary()

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model.
with open(model_path, "wb") as f:
    f.write(tflite_model)

In [ ]:
!zip submission.zip $model_path